# Path function

The goal of the MADlib path function is to perform regular pattern matching over a sequence of rows, and to extract useful information about the pattern matches. The useful information could be a simple count of matches or something more involved like aggregations or window functions.

In [1]:
%load_ext sql

In [2]:
# Greenplum Database 5.x on GCP (PM demo machine) - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

In [3]:
%sql select madlib.version();

1 rows affected.


version
"MADlib version: 1.18.0-dev, git revision: rel/v1.17.0-100-g4987e8f, cmake configuration time: Wed Mar 24 23:51:47 UTC 2021, build type: release, build system: Linux-3.10.0-1160.21.1.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


The data set describes shopper behavior on a notional web site that sells beer and wine. A beacon fires an event to a log file when the shopper visits different pages on the site: landing page, beer selection page, wine selection page, and checkout. Other pages on the site like help pages show up in the logs as well. Let’s assume that the log has been sessionized.

In [4]:
%%sql 
DROP TABLE IF EXISTS eventlog, path_output, path_output_tuples CASCADE;
CREATE TABLE eventlog (event_timestamp TIMESTAMP,
            user_id INT,
            session_id INT,
            page TEXT,
            revenue FLOAT);
INSERT INTO eventlog VALUES
('04/15/2015 01:03:00', 100821, 100, 'LANDING', 0),
('04/15/2015 01:04:00', 100821, 100, 'WINE', 0),
('04/15/2015 01:05:00', 100821, 100, 'CHECKOUT', 39),
('04/15/2015 02:06:00', 100821, 101, 'WINE', 0),
('04/15/2015 02:09:00', 100821, 101, 'WINE', 0),
('04/15/2015 01:15:00', 101121, 102, 'LANDING', 0),
('04/15/2015 01:16:00', 101121, 102, 'WINE', 0),
('04/15/2015 01:17:00', 101121, 102, 'CHECKOUT', 15),
('04/15/2015 01:18:00', 101121, 102, 'LANDING', 0),
('04/15/2015 01:19:00', 101121, 102, 'HELP', 0),
('04/15/2015 01:21:00', 101121, 102, 'WINE', 0),
('04/15/2015 01:22:00', 101121, 102, 'CHECKOUT', 23),
('04/15/2015 02:15:00', 101331, 103, 'LANDING', 0),
('04/15/2015 02:16:00', 101331, 103, 'WINE', 0),
('04/15/2015 02:17:00', 101331, 103, 'HELP', 0),
('04/15/2015 02:18:00', 101331, 103, 'WINE', 0),
('04/15/2015 02:19:00', 101331, 103, 'CHECKOUT', 16),
('04/15/2015 02:22:00', 101443, 104, 'BEER', 0),
('04/15/2015 02:25:00', 101443, 104, 'CHECKOUT', 12),
('04/15/2015 02:29:00', 101881, 105, 'LANDING', 0),
('04/15/2015 02:30:00', 101881, 105, 'BEER', 0),
('04/15/2015 01:05:00', 102201, 106, 'LANDING', 0),
('04/15/2015 01:06:00', 102201, 106, 'HELP', 0),
('04/15/2015 01:09:00', 102201, 106, 'LANDING', 0),
('04/15/2015 02:15:00', 102201, 107, 'WINE', 0),
('04/15/2015 02:16:00', 102201, 107, 'BEER', 0),
('04/15/2015 02:17:00', 102201, 107, 'WINE', 0),
('04/15/2015 02:18:00', 102871, 108, 'BEER', 0),
('04/15/2015 02:19:00', 102871, 108, 'WINE', 0),
('04/15/2015 02:22:00', 102871, 108, 'CHECKOUT', 21),
('04/15/2015 02:25:00', 102871, 108, 'LANDING', 0),
('04/15/2015 02:17:00', 103711, 109, 'BEER', 0),
('04/15/2015 02:18:00', 103711, 109, 'LANDING', 0),
('04/15/2015 02:19:00', 103711, 109, 'WINE', 0);

SELECT * FROM eventlog ORDER BY event_timestamp ASC;

Done.
Done.
34 rows affected.
34 rows affected.


event_timestamp,user_id,session_id,page,revenue
2015-04-15 01:03:00,100821,100,LANDING,0.0
2015-04-15 01:04:00,100821,100,WINE,0.0
2015-04-15 01:05:00,102201,106,LANDING,0.0
2015-04-15 01:05:00,100821,100,CHECKOUT,39.0
2015-04-15 01:06:00,102201,106,HELP,0.0
2015-04-15 01:09:00,102201,106,LANDING,0.0
2015-04-15 01:15:00,101121,102,LANDING,0.0
2015-04-15 01:16:00,101121,102,WINE,0.0
2015-04-15 01:17:00,101121,102,CHECKOUT,15.0
2015-04-15 01:18:00,101121,102,LANDING,0.0


Calculate the revenue by checkout:

In [5]:
%%sql
SELECT madlib.path(
     'eventlog',                -- Name of input table
     'path_output',             -- Table name to store path results
     'session_id',              -- Partition input table by session
     'event_timestamp ASC',     -- Order partitions in input table by time
     'buy:=page=''CHECKOUT''',  -- Define a symbol for checkout events
     '(buy)',                   -- Pattern search: purchase
     'sum(revenue) as checkout_rev',    -- Aggregate:  sum revenue by checkout
     TRUE                       -- Persist matches
     );
SELECT * FROM path_output ORDER BY session_id, match_id;

1 rows affected.
6 rows affected.


session_id,match_id,checkout_rev
100,1.0,39.0
102,1.0,15.0
102,2.0,23.0
103,1.0,16.0
104,1.0,12.0
108,1.0,21.0


Note that there are 2 checkouts within session 102, which is apparent from the 'match_id' column. This serves to illustrate that the 'aggregate_func' operates on a per pattern match basis, not on a per partition basis. If in fact we wanted revenue by partition ('session_id' in this example), then we could do:

In [6]:
%%sql
SELECT session_id, sum(checkout_rev) FROM path_output GROUP BY session_id ORDER BY session_id;

5 rows affected.


session_id,sum
100,39.0
102,38.0
103,16.0
104,12.0
108,21.0


Since we set TRUE for 'persist_rows', we can view the associated pattern matches:

In [7]:
%%sql
SELECT * FROM path_output_tuples ORDER BY session_id ASC, event_timestamp ASC;

6 rows affected.


event_timestamp,user_id,session_id,page,revenue,symbol,match_id
2015-04-15 01:05:00,100821,100,CHECKOUT,39.0,buy,1.0
2015-04-15 01:17:00,101121,102,CHECKOUT,15.0,buy,1.0
2015-04-15 01:22:00,101121,102,CHECKOUT,23.0,buy,2.0
2015-04-15 02:19:00,101331,103,CHECKOUT,16.0,buy,1.0
2015-04-15 02:25:00,101443,104,CHECKOUT,12.0,buy,1.0
2015-04-15 02:22:00,102871,108,CHECKOUT,21.0,buy,1.0


Notice that the 'symbol' and 'match_id' columns are added to the right of the matched rows.

We are interested in sessions with an order placed within 4 pages of entering the shopping site via the landing page. We represent this by the regular expression: '(land)[^(land)(buy)]{0,2}(buy)'. In other words, visit to the landing page followed by from 0 to 2 non-entry, non-sale pages, followed by a purchase. The SQL is as follows:

In [8]:
%%sql
DROP TABLE IF EXISTS path_output, path_output_tuples;
SELECT madlib.path(
     'eventlog',                -- Name of input table
     'path_output',             -- Table name to store path results
     'session_id',              -- Partition input table by session
     'event_timestamp ASC',     -- Order partitions in input table by time
      $$ land:=page='LANDING',
        wine:=page='WINE',
        beer:=page='BEER',
        buy:=page='CHECKOUT',
        other:=page<>'LANDING' AND page<>'WINE' AND page<>'BEER' AND  page<>'CHECKOUT'
        $$,                     -- Symbols for page types

      '(land)[^(land)(buy)]{0,2}(buy)', -- Purchase within 4 pages entering site
     'sum(revenue) as checkout_rev',    -- Aggregate:  sum revenue by checkout
     TRUE                       -- Persist matches
     );
SELECT * FROM path_output ORDER BY session_id, match_id;

Done.
1 rows affected.
3 rows affected.


session_id,match_id,checkout_rev
100,1.0,39.0
102,1.0,15.0
102,2.0,23.0


Now view the associated pattern matches:

In [9]:
%%sql
SELECT * FROM path_output_tuples ORDER BY session_id ASC, event_timestamp ASC;

10 rows affected.


event_timestamp,user_id,session_id,page,revenue,symbol,match_id
2015-04-15 01:03:00,100821,100,LANDING,0.0,land,1.0
2015-04-15 01:04:00,100821,100,WINE,0.0,wine,1.0
2015-04-15 01:05:00,100821,100,CHECKOUT,39.0,buy,1.0
2015-04-15 01:15:00,101121,102,LANDING,0.0,land,1.0
2015-04-15 01:16:00,101121,102,WINE,0.0,wine,1.0
2015-04-15 01:17:00,101121,102,CHECKOUT,15.0,buy,1.0
2015-04-15 01:18:00,101121,102,LANDING,0.0,land,2.0
2015-04-15 01:19:00,101121,102,HELP,0.0,other,2.0
2015-04-15 01:21:00,101121,102,WINE,0.0,wine,2.0
2015-04-15 01:22:00,101121,102,CHECKOUT,23.0,buy,2.0


For instances where a purchase is made within 4 pages of entering a site, compute the elasped time to checkout:

In [10]:
%%sql
DROP TABLE IF EXISTS path_output, path_output_tuples;
SELECT madlib.path(
     'eventlog',                -- Name of input table
     'path_output',             -- Table name to store path results
     'session_id',              -- Partition input table by session
     'event_timestamp ASC',     -- Order partitions in input table by time
      $$ land:=page='LANDING',
        wine:=page='WINE',
        beer:=page='BEER',
        buy:=page='CHECKOUT',
        other:=page<>'LANDING' AND page<>'WINE' AND page<>'BEER' AND  page<>'CHECKOUT'
        $$,                     -- Symbols for page types
      '(land)[^(land)(buy)]{0,2}(buy)', -- Purchase within 4 pages entering site
     '(max(event_timestamp)-min(event_timestamp)) as elapsed_time',    -- Aggregate: elapsed time
     TRUE                       -- Persist matches
     );
SELECT * FROM path_output ORDER BY session_id, match_id;


Done.
1 rows affected.
3 rows affected.


session_id,match_id,elapsed_time
100,1.0,0:02:00
102,1.0,0:02:00
102,2.0,0:04:00


We may want to use a window function instead of an aggregate. You can write window functions on the output tuples to achieve the desired result.   Continuing the previous example, let’s say we want to compute average revenue for checkouts within 4 pages of entering the shopping site via the landing page:

In [11]:
%%sql
SELECT DATE(event_timestamp), user_id, session_id, revenue,
    avg(revenue) OVER (PARTITION BY DATE(event_timestamp)) as avg_checkout_rev
    FROM path_output_tuples
    WHERE page='CHECKOUT'
    ORDER BY user_id, session_id;

3 rows affected.


date,user_id,session_id,revenue,avg_checkout_rev
2015-04-15,100821,100,39.0,25.6666666667
2015-04-15,101121,102,15.0,25.6666666667
2015-04-15,101121,102,23.0,25.6666666667


Now we want to do a golden path analysis to find the most successful shopper paths through the site. Since our data set is small, we decide this means the most frequently viewed page just before a checkout is made:

In [12]:
%%sql
DROP TABLE IF EXISTS path_output, path_output_tuples;
SELECT madlib.path(
     'eventlog',                -- Name of input table
     'path_output',             -- Table name to store path results
     'session_id',              -- Partition input table by session
     'event_timestamp ASC',     -- Order partitions in input table by time
      $$ land:=page='LANDING',
        wine:=page='WINE',
        beer:=page='BEER',
        buy:=page='CHECKOUT',
        other:=page<>'LANDING' AND page<>'WINE' AND page<>'BEER' AND  page<>'CHECKOUT'
        $$,                     -- Symbols for page types
      '[^(buy)](buy)',          -- Pattern to match
     'array_agg(page ORDER BY session_id ASC, event_timestamp ASC) as page_path');
     
SELECT count(*), page_path from
    (SELECT * FROM path_output) q
GROUP BY page_path
ORDER BY count(*) DESC
LIMIT 10;

Done.
1 rows affected.
2 rows affected.


count,page_path
5,"[u'WINE', u'CHECKOUT']"
1,"[u'BEER', u'CHECKOUT']"


There are only 2 different paths. The wine page is viewed more frequently than the beer page just before checkout.

To demonstrate the use of 'overlapping_patterns', consider a pattern with at least one page followed by and ending with a checkout:

In [13]:
%%sql
DROP TABLE IF EXISTS path_output, path_output_tuples;
SELECT madlib.path(                                                                   
     'eventlog',                    -- Name of the table                                           
     'path_output',                 -- Table name to store the path results                         
     'session_id',                  -- Partition by session                 
     'event_timestamp ASC',         -- Order partitions in input table by time       
     $$ nobuy:=page<>'CHECKOUT',
        buy:=page='CHECKOUT'
     $$,  -- Definition of symbols used in the pattern definition 
     '(nobuy)+(buy)',         -- At least one page followed by and ending with a CHECKOUT.
     'array_agg(page ORDER BY session_id ASC, event_timestamp ASC) as page_path',  
     FALSE,                        -- Don't persist matches
     TRUE                          -- Turn on overlapping patterns
     );
SELECT * FROM path_output ORDER BY session_id, match_id;

Done.
1 rows affected.
14 rows affected.


session_id,match_id,page_path
100,1.0,"[u'LANDING', u'WINE', u'CHECKOUT']"
100,2.0,"[u'WINE', u'CHECKOUT']"
102,1.0,"[u'LANDING', u'WINE', u'CHECKOUT']"
102,2.0,"[u'WINE', u'CHECKOUT']"
102,3.0,"[u'LANDING', u'HELP', u'WINE', u'CHECKOUT']"
102,4.0,"[u'HELP', u'WINE', u'CHECKOUT']"
102,5.0,"[u'WINE', u'CHECKOUT']"
103,1.0,"[u'LANDING', u'WINE', u'HELP', u'WINE', u'CHECKOUT']"
103,2.0,"[u'WINE', u'HELP', u'WINE', u'CHECKOUT']"
103,3.0,"[u'HELP', u'WINE', u'CHECKOUT']"


With overlap turned off, the result is:

In [14]:
%%sql
DROP TABLE IF EXISTS path_output, path_output_tuples;
SELECT madlib.path(                                                                   
     'eventlog',                    -- Name of the table                                           
     'path_output',                 -- Table name to store the path results                         
     'session_id',                  -- Partition by session                 
     'event_timestamp ASC',         -- Order partitions in input table by time       
     $$ nobuy:=page<>'CHECKOUT',
        buy:=page='CHECKOUT'
     $$,  -- Definition of symbols used in the pattern definition 
     '(nobuy)+(buy)',         -- At least one page followed by and ending with a CHECKOUT.
     'array_agg(page ORDER BY session_id ASC, event_timestamp ASC) as page_path',  
     FALSE,                        -- Don't persist matches
     FALSE                          -- Turn on overlapping patterns
     );
SELECT * FROM path_output ORDER BY session_id, match_id;

Done.
1 rows affected.
6 rows affected.


session_id,match_id,page_path
100,1.0,"[u'LANDING', u'WINE', u'CHECKOUT']"
102,1.0,"[u'LANDING', u'WINE', u'CHECKOUT']"
102,2.0,"[u'LANDING', u'HELP', u'WINE', u'CHECKOUT']"
103,1.0,"[u'LANDING', u'WINE', u'HELP', u'WINE', u'CHECKOUT']"
104,1.0,"[u'BEER', u'CHECKOUT']"
108,1.0,"[u'BEER', u'WINE', u'CHECKOUT']"
